In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [2]:
img_labels = []
directory_path = Path('YogaPoses')
for file_path in directory_path.glob('*'):
    img_labels.append(str(file_path).split('\\')[-1])
    
labels_to_digits = {}
digits_to_labels = {}
for i, elem in enumerate(img_labels):
    labels_to_digits[elem] = i
    digits_to_labels[i] = elem
labels_to_digits, digits_to_labels

({'Downdog': 0, 'Goddess': 1, 'Plank': 2, 'Tree': 3, 'Warrior2': 4},
 {0: 'Downdog', 1: 'Goddess', 2: 'Plank', 3: 'Tree', 4: 'Warrior2'})

In [3]:
img_path = 'YogaPoses/'

# Словарь с названиями ключевых точек тела в MediaPipe Pose
# Заметьте, что точная структура может меняться в зависимости от версии MediaPipe, поэтому проверьте актуальность данных
pose_landmarks = {
    0: 'Нос',
    1: 'Внутренний глаз слева',
    2: 'Внутренний глаз справа',
    3: 'Внешний глаз слева',
    4: 'Внешний глаз справа',
    5: 'Левое ухо',
    6: 'Правое ухо',
    7: 'Рот слева',
    8: 'Рот справа',
    9: 'Левое плечо',
    10: 'Правое плечо',
    11: 'Левый локоть',
    12: 'Правый локоть',
    13: 'Левое запястье',
    14: 'Правое запястье',
    15: 'Левая ладонь',
    16: 'Правая ладонь',
    17: 'Левый мизинец',
    18: 'Правый мизинец',
    19: 'Левый указательный палец',
    20: 'Правый указательный палец',
    21: 'Левый большой палец',
    22: 'Правый большой палец',
    23: 'Левый бедро',
    24: 'Правое бедро',
    25: 'Левое колено',
    26: 'Правое колено',
    27: 'Левая щиколотка',
    28: 'Правая щиколотка',
    29: 'Левая пятка',
    30: 'Правая пятка',
    31: 'Левый передний край стопы',
    32: 'Правый передний край стопы',
}

In [4]:
directory_path = img_path

for elem in img_labels:
    if not os.path.isdir(directory_path+elem):
        print(f'Directory {directory_path+elem} is not exist')

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def work_with_file(file_path, df, label):
    img = cv2.imread(file_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)
    if results.pose_landmarks:
        landmarks = {'image_path': str(file_path)}
        
        for i, landmark in enumerate(results.pose_landmarks.landmark):
            landmarks[f'x_{i}'] = landmark.x
            landmarks[f'y_{i}'] = landmark.y
            landmarks[f'z_{i}'] = landmark.z
            landmarks[f'visibility_{i}'] = landmark.visibility
        landmarks['label'] = labels_to_digits[label]
        
        df = df._append(landmarks, ignore_index=True)
    
    return df

In [8]:
df = pd.DataFrame()
for elem in img_labels:      
    n_files = 0
    directory_path = Path(img_path + elem)

    for file_path in directory_path.glob('*'):
        if file_path.is_file():
            n_files += 1
            df = work_with_file(str(file_path), df, elem)
            
    print(f'got {n_files} files from {directory_path} folder')
    
    
df.to_csv('body_landmarks.csv', index=False)
df

got 196 files from YogaPoses\Downdog folder
got 199 files from YogaPoses\Goddess folder
got 197 files from YogaPoses\Plank folder
got 198 files from YogaPoses\Tree folder
got 198 files from YogaPoses\Warrior2 folder


,image_path,x_0,y_0,z_0,visibility_0,x_1,y_1,z_1,visibility_1,x_2,...,visibility_30,x_31,y_31,z_31,visibility_31,x_32,y_32,z_32,visibility_32,label
0,YogaPoses\Downdog\00000000.jpg,0.508115,0.713662,-0.053362,0.999236,0.492659,0.738126,-0.091488,0.999589,0.489574,...,0.574398,0.799371,0.925598,-0.253609,0.979984,0.777966,0.902165,0.210656,0.578596,0
1,YogaPoses\Downdog\00000001.jpg,0.568524,0.787049,-0.057612,0.999562,0.582195,0.793212,-0.039310,0.999590,0.583312,...,0.996091,0.249844,0.895454,0.122632,0.686935,0.237913,0.903858,-0.221486,0.992437,0
2,YogaPoses\Downdog\00000002.jpg,0.611821,0.728562,-0.036824,0.999567,0.632072,0.732158,-0.011005,0.999586,0.633908,...,0.995154,0.208138,0.899928,0.218623,0.615439,0.200240,0.915475,-0.311037,0.990648,0
3,YogaPoses\Downdog\00000003.jpg,0.542666,0.760745,-0.090946,0.999914,0.564063,0.771601,-0.071252,0.999896,0.566818,...,0.997554,0.275256,0.874784,0.345119,0.822172,0.262187,0.879788,-0.103446,0.995908,0
4,YogaPoses\Downdog\00000004.jpg,0.360084,0.698306,-0.045246,0.997917,0.338686,0.709288,-0.087716,0.998730,0.335440,...,0.479459,0.740724,0.922454,-0.222143,0.971605,0.723063,0.911414,0.317072,0.485925,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,YogaPoses\Warrior2\00000433.jpg,0.560225,0.140472,-0.234602,0.999982,0.570026,0.117040,-0.224115,0.999924,0.573986,...,0.986391,0.804593,0.974238,0.035640,0.979761,0.402176,0.961504,-0.257184,0.992917,4
975,YogaPoses\Warrior2\00000434.jpg,0.554509,0.422001,-0.047373,0.999989,0.559316,0.411220,-0.054730,0.999959,0.560208,...,0.988508,0.784045,0.862466,-0.294666,0.998275,0.493387,0.875531,0.047510,0.990512,4
976,YogaPoses\Warrior2\00000436.jpg,0.426025,0.540080,-0.282692,0.999829,0.436883,0.534927,-0.278971,0.998497,0.440015,...,0.996489,0.595916,0.853134,0.060400,0.995538,0.219673,0.795636,-0.051350,0.997210,4
977,YogaPoses\Warrior2\00000437.jpg,0.449105,0.177476,-0.392941,0.999945,0.462741,0.158149,-0.380931,0.999660,0.468977,...,0.990126,0.825684,0.917848,-0.106795,0.980022,0.198446,0.916430,-0.155226,0.989153,4
